#### Notebook 'Tempo'

In [43]:
# Carga de librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from itertools import combinations
from collections import Counter
import networkx as nx
from pyvis.network import Network


In [44]:
# Crear el DataFrame 'df_aspectos' con los aspectos y las variables asociadas
data_aspectos = {
    'aspecto': ['DROGAS_Y_CORRUPCIÓN'],
    'var_1': ['per603'],
    'var_2': ['per604'],
    'var_3': ['per605'],
    'var_4': ['per104'],
    'var_5': ['per304']
}
df_aspectos = pd.DataFrame(data_aspectos)
display(df_aspectos)
# cargar Analytical_Data
df_AD = pd.read_excel('data/Analytical_Data.xlsx')
display(df_AD.head())

,aspecto,var_1,var_2,var_3,var_4,var_5
0,DROGAS_Y_CORRUPCIÓN,per603,per604,per605,per104,per304


,Dominio,Nombre_Dominio,Variable_per,Detalle_Dominio
0,Domain_1,External Relations,per101,Foreign Special Relationships: Positive
1,Domain_1,External Relations,per102,Foreign Special Relationships: Negative
2,Domain_1,External Relations,per103,Anti-Imperialism
3,Domain_1,External Relations,per104,Military: Positive
4,Domain_1,External Relations,per105,Military: Negative


In [45]:
# Cargar datasets
df = pd.read_excel('./data/MP_Dataset_KMeans_mean.xlsx')  # Dataset principal
print("Datos cargados:")
display(df.head())


Datos cargados:


,agno,countryname,per101,per102,per103,per104,per105,per106,per107,per108,...,per6013,per6014,per6061,per6071,per6072,per6081,per7051,per7052,per7061,per7062
0,1997,Albania,1.770988,0.00000,0.031388,3.046506,0.015910,0.032313,3.406129,1.114989,...,2.021481,0.0,0.0,0.0,0.0,0.0,0.637892,0.363756,0.0,0.0
1,2001,Albania,2.392570,0.00000,0.000000,2.757403,0.026619,0.023948,3.523162,1.452779,...,0.989546,0.0,0.0,0.0,0.0,0.0,0.897238,0.508457,0.0,0.0
2,1989,Argentina,0.000000,0.00000,4.619213,0.448618,0.828036,0.338070,2.955090,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
3,1995,Argentina,0.061642,0.02014,0.143247,0.882433,0.083005,1.107286,1.996025,1.669947,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0
4,1999,Argentina,0.192543,0.00000,0.582363,0.524400,0.017829,0.613450,3.482869,2.262154,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0


#### Paso 3: Definición de Funciones

##### Filtrar y Normalizar

In [47]:
def filtrar_normalizar(df_periodo, variables):
    df_filtered = df_periodo[['countryname', 'agno'] + list(variables)].copy()
    df_filtered.dropna(inplace=True)
    columns_per = variables
    df_filtered['per_sum'] = df_filtered[columns_per].sum(axis=1)
    rows_to_normalize = df_filtered['per_sum'] != 1.0
    df_filtered.loc[rows_to_normalize, columns_per] = df_filtered.loc[rows_to_normalize, columns_per].div(
        df_filtered.loc[rows_to_normalize, 'per_sum'], axis=0
    )
    df_filtered.drop(columns=['per_sum'], inplace=True)
    return df_filtered


In [48]:
def preprocesar_periodo(df, desde, hasta):
    """
    Preprocesa el DataFrame para el período seleccionado.
    Combina registros de países con múltiples elecciones en el período y normaliza las variables.
    
    Parámetros:
    - df: DataFrame principal con datos de programas de gobierno.
    - desde: Año inicial del período.
    - hasta: Año final del período.

    Retorna:
    - df_periodo: DataFrame preprocesado con un registro por país en el período.
    """
    # Filtrar el DataFrame para el período
    df_periodo = df[(df['agno'] >= desde) & (df['agno'] <= hasta)]

    # Identificar países con más de una elección en el período
    duplicated_countries = df_periodo['countryname'].value_counts()
    duplicated_countries = duplicated_countries[duplicated_countries > 1]

    if not duplicated_countries.empty:
        print("\nPaíses con más de una elección en el período:")
        for country in duplicated_countries.index:
            print(f"\n{country}:")

            # Filtrar datos del país con duplicados
            country_data = df_periodo[df_periodo['countryname'] == country]

            # Imprimir los registros originales
            print(f"Registros originales ({len(country_data)}):")
            display(country_data)

            # Promediar las columnas `per` y conservar el primer año
            per_columns = [col for col in df_periodo.columns if col.startswith('per')]
            averaged_row = country_data[per_columns].mean().to_dict()
            averaged_row['agno'] = country_data['agno'].min()  # Primer año del período
            averaged_row['countryname'] = country

            # Imprimir los valores promedio
            print("\nValores promediados:")
            for key, value in averaged_row.items():
                if key in per_columns:  # Solo mostrar columnas 'per'
                    print(f"{key}: {value:.4f}")

            # Eliminar los registros originales del dataframe filtrado
            df_periodo = df_periodo[df_periodo['countryname'] != country]

            # Agregar el registro promediado al dataframe
            df_periodo = pd.concat([df_periodo, pd.DataFrame([averaged_row])], ignore_index=True)

    else:
        print("\nNo hay países con más de una elección en el período.")

    # Verificar los resultados finales
    print("\nDataframe ajustado (un registro por país en el período):")
    display(df_periodo)

    return df_periodo


In [49]:
def filtrar_y_normalizar(df_periodo, variables):
    """
    Filtra y normaliza las columnas 'per' en un DataFrame para un período específico.

    Parámetros:
    - df_periodo: DataFrame ya preprocesado para un período (salida de `preprocesar_periodo`).
    - variables: Lista de columnas 'per' relevantes para el aspecto en análisis.

    Retorna:
    - df_filtered: DataFrame filtrado y normalizado.
    """
    # Filtrar el DataFrame para las columnas seleccionadas
    df_filtered = df_periodo[['countryname', 'agno'] + list(variables)].copy()
    df_filtered.dropna(inplace=True)  # Eliminar filas con valores NaN

    # Verificar si hay filas eliminadas
    if df_periodo.isna().sum().sum() > 0:
        print("Se encontraron valores NaN que fueron eliminados.")

    # Normalizar las columnas 'per'
    columns_per = variables
    df_filtered['per_sum'] = df_filtered[columns_per].sum(axis=1)
    rows_to_normalize = df_filtered['per_sum'] != 1.0
    df_filtered.loc[rows_to_normalize, columns_per] = df_filtered.loc[rows_to_normalize, columns_per].div(
        df_filtered.loc[rows_to_normalize, 'per_sum'], axis=0
    )
    df_filtered.drop(columns=['per_sum'], inplace=True)

    # Verificar que las variables 'per' sumen 1
    assert df_filtered[columns_per].sum(axis=1).round(6).eq(1).all(), "Error: las variables 'per' no suman 1."

    return df_filtered


##### PCA y K-MEANS

In [50]:
def ejecutar_y_visualizar_pca_kmeans(df_filtered, variables):
    """
    Ejecuta PCA y K-MEANS, y visualiza los resultados en 2D y 3D.
    
    Parámetros:
    - df_filtered: DataFrame filtrado y normalizado.
    - variables: Lista de variables 'per' a incluir en el análisis.
    
    Retorna:
    - df_pca: DataFrame con las componentes principales y los clústeres.
    - pca: Objeto PCA ajustado.
    - kmeans: Objeto K-MEANS ajustado.
    """
    # Ejecutar PCA y K-MEANS
    df_pca, pca, kmeans = ejecutar_pca_kmeans(df_filtered, variables)
    
    # Verificar resultados del PCA
    print("Resultados del PCA:")
    print(df_pca.info())
    display(df_pca.head())
    
    # Visualizar los clústeres en 2D
    visualizar_clusters(df_pca, kmeans)
    
    return df_pca, pca, kmeans


##### Matriz de coincidencias

In [51]:
def construir_matriz_coincidencias(resultados_temporales):
    coincidencias = Counter()
    for clusters in resultados_temporales.values():
        for cluster in clusters.values():
            for pair in combinations(sorted(cluster), 2):
                coincidencias[pair] += 1

    # Convertir a matriz
    paises = sorted(set([p for pair in coincidencias.keys() for p in pair]))
    coincidence_matrix = pd.DataFrame(0, index=paises, columns=paises)

    for (p1, p2), count in coincidencias.items():
        coincidence_matrix.loc[p1, p2] = count
        coincidence_matrix.loc[p2, p1] = count

    return coincidence_matrix


##### Visualización del grafo

In [52]:
def grafo_coincidencias(coincidence_matrix):
    # Crear el grafo
    G = nx.Graph()
    for i, row in coincidence_matrix.iterrows():
        for j, value in row.iteritems():
            if value > 0:  # Agregar conexiones con peso mayor a 0
                G.add_edge(i, j, weight=value)

    # Convertir a grafo interactivo con PyVis
    net = Network(notebook=True, height="750px", width="100%")
    net.from_nx(G)
    net.show("grafo_coincidencias.html")


#### Ejecución para un período

In [ ]:
# Definir el período específico
desde = 2019
hasta = 2022

# Preparar y normalizar los datos del período seleccionado
df_periodo = preprocesar_periodo(df, desde, hasta)
df_filtered = filtrar_y_normalizar(df_periodo, variables)

# Verificar el resultado del DataFrame filtrado
print("\nDataFrame filtrado y normalizado:")
print(df_filtered.info())
display(df_filtered.head())

# Ejecutar y visualizar PCA y K-MEANS
df_pca, pca, kmeans = ejecutar_y_visualizar_pca_kmeans(df_filtered, variables)


#### Ejecución para Múltiples Períodos

In [ ]:
# Definir períodos múltiples para el análisis
periodos = [(1970, 1972), (1973, 1975), (1976, 1978), (2019, 2022)]

# Diccionario para almacenar resultados por período
resultados_temporales = {}

for desde, hasta in periodos:
    print(f"\nProcesando período: {desde}-{hasta}")
    
    # Preprocesar y normalizar datos
    df_periodo = preprocesar_periodo(df, desde, hasta)
    df_filtered = filtrar_y_normalizar(df_periodo, variables)
    
    # Ejecutar PCA y K-MEANS
    df_pca, pca, kmeans = ejecutar_y_visualizar_pca_kmeans(df_filtered, variables)
    
    # Almacenar los países en cada clúster
    clusters = {}
    for cluster in df_pca['cluster'].unique():
        clusters[cluster] = df_pca[df_pca['cluster'] == cluster]['countryname'].unique()
    resultados_temporales[f"{desde}-{hasta}"] = clusters

# Construir y visualizar la matriz de coincidencias
coincidence_matrix = construir_matriz_coincidencias(resultados_temporales)
grafo_coincidencias(coincidence_matrix)
